In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install albumentations==0.4.6
import albumentations 
pip install patichify 
pip install SimpleITK

In [2]:

import sys
import random



from tensorflow.keras.optimizers import Adam

from datetime import datetime 

from PIL import Image
from keras import backend, optimizers

import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
#from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from matplotlib import pyplot as plt
import os, glob
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
import imageio
from albumentations import HorizontalFlip, VerticalFlip,CenterCrop, Rotate,RandomContrast,Transpose
from tqdm import tqdm
import imageio
import numpy as np
from skimage.io import imread_collection

#import SimpleITK as sitk

MIP for image and label

In [ ]:
dir = 'content/drive/MyDrive/ADAM-Filtered-Data/'
output='/content/drive/MyDrive/Slices'
for index,file_name in enumerate(sorted(os.listdir('/content/drive/MyDrive/ADAM-Filtered-Data'))):
  if file_name.endswith('TOF.nii.gz'):
    image=sitk.ReadImage('/content/drive/MyDrive/ADAM-Filtered-Data/' + file_name)
    mip_axial = sitk.MaximumProjection(image, projectionDimension=2)[:,:,0]
    #sitk.WriteImage(sitk.Cast(sitk.RescaleIntensity(mip_axial),sitk.sitkUInt8),output, f'axialMIP{index}.png')
    sitk.WriteImage(sitk.Cast(sitk.RescaleIntensity(mip_axial),sitk.sitkUInt8),os.path.join(output, f'axialMIPimage{index}.png') )

dir = 'content/drive/MyDrive/ADAM-Filtered-Data/'
output2='/content/drive/MyDrive/Mask'
for index,file_name in enumerate(sorted(os.listdir('/content/drive/MyDrive/ADAM-Filtered-Data'))):
  if file_name.endswith('aneurysms.nii.gz'):
    image=sitk.ReadImage('/content/drive/MyDrive/ADAM-Filtered-Data/' + file_name)
    mip_axial = sitk.MaximumProjection(image, projectionDimension=2)[:,:,0]
    #sitk.WriteImage(sitk.Cast(sitk.RescaleIntensity(mip_axial),sitk.sitkUInt8),output, f'axialMIP{index}.png')
    sitk.WriteImage(sitk.Cast(sitk.RescaleIntensity(mip_axial),sitk.sitkUInt8),os.path.join(output2, f'axialMIPMask{index}.png') )

resizing test data

In [ ]:
train_x=sorted(os.listdir('/content/drive/MyDrive/realpatch/test/images/'))
train_y=sorted(os.listdir('/content/drive/MyDrive/realpatch/test/masks/'))


#set size
size=(256,256)
  
for x,y in tqdm(zip(train_x, train_y)):
    #avoid overwritting 
    name=x.split('.')[0]
    #read image 
    x=cv2.imread('/content/drive/MyDrive/realpatch/test/images/'+x)
    y=cv2.imread('/content/drive/MyDrive/realpatch/test/masks/'+y)
#reszie image and mask
    x=cv2.resize(x,size)
    y=cv2.resize(y,size)

    tmp_image_name=f'{name}.png'
      
    tmp_mask_name=f'{name}.png'
#save images masks
    cv2.imwrite('/content/drive/MyDrive/realpatch/test/newtest/images/'+tmp_image_name, x)
    cv2.imwrite("/content/drive/MyDrive/realpatch/test/newtest/masks/"+tmp_mask_name, y)

data augmentation 

In [ ]:
#create directory
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)
#load iamges and masks
def load_data(path):
  train_x=sorted(os.listdir(path+'/images/'))
  train_y=sorted(os.listdir(path+"/masks/"))

  

  return (train_x, train_y)

def augument_data(images, masks, save_path, augument=True):
  size=(256,256)
  
  for x,y in tqdm((zip(images,masks)), total=len(images)):
    
    name=x.split('.')[0]
    x=cv2.imread(data_path+'/images/'+x)
    y=cv2.imread(data_path+"/masks/"+y)

    if augument == True:
      
      aug=HorizontalFlip(p=1.0)
      augumented=aug(image=x, mask=y)
      x1=augumented['image']
      y1=augumented['mask']

      aug=VerticalFlip(p=1.0)
      augumented=aug(image=x, mask=y)
      x2=augumented['image']
      y2=augumented['mask']

     
      aug=RandomContrast(limit=0.2, always_apply=True, p=0.5)
      augumented=aug(image=x, mask=y)
      x3=augumented['image']
      y3=augumented['mask']

      aug=Rotate(limit=20, p=1.0)
      augumented=aug(image=x, mask=y)
      x4=augumented['image']
      y4=augumented['mask']

      aug=Transpose(p=1)
      augumented=aug(image=x, mask=y)
      x5=augumented['image']
      y5=augumented['mask']

    


      aug=Rotate(limit=35, p=1.0)
      augumented=aug(image=x, mask=y)
      x6=augumented['image']
      y6=augumented['mask']

     

      aug=Rotate(limit=75, p=1.0)
      augumented=aug(image=x, mask=y)
      x7=augumented['image']
      y7=augumented['mask']

      aug=Rotate(limit=55, p=1.0)
      augumented=aug(image=x, mask=y)
      x8=augumented['image']
      y8=augumented['mask']

      aug=Rotate(limit=95, p=1.0)
      augumented=aug(image=x, mask=y)
      x9=augumented['image']
      y9=augumented['mask']
    

 
      X=[x,x1,x2,x3,x4,x5,x6,x7,x8,x9]
      Y=[y,y1,y2,y3,y4,y5,y6,y7,y8,y9]
      
      
    else:
      X=[x]
      Y=[y]
    
    for index, (i, m) in enumerate(zip(X,Y)):
      i=cv2.resize(i,size)
      m=cv2.resize(m,size) 

      tmp_image_name=f'{name}_{index}.png'
      tmp_mask_name=f'{name}_{index}.png'


      

      cv2.imwrite(data_path+'new/images/'+tmp_image_name, i)
      cv2.imwrite(data_path+'new/masks/'+tmp_mask_name, m)

if __name__=='__main__':
  np.random.seed(42)
  #Specify the path
  data_path = "/content/drive/MyDrive/realpatch/train/new3/"
  (train_x, train_y)=load_data(data_path)

  #Directories for augumented data
  create_dir(data_path+"new/images/")
  create_dir(data_path+"new/masks/")
  
 #create save new images
  augument_data(train_x, train_y,'/content/drive/MyDrive/realpatch/train/new3/new/', augument=True)


  


extract patches

In [ ]:
large_image_stack=sorted(os.listdir('/content/drive/MyDrive/MRA/traincrop/images/'))
large_mask_stack=sorted(os.listdir('/content/drive/MyDrive/MRA/traincrop/masks/'))
for x in large_image_stack:
  name=x.split('.')[0]
  #read images
  img = cv2.imread("/content/drive/MyDrive/MRA/traincrop/masks/"+x)
  #use patchify library to extraxt patches 
  #patch size 64 overlap in middle (32)
  patches_img = patchify(img, (64,64,3), step=32)
  tmp_image_name=f'{name}'
  for i in range(patches_img.shape[0]):
      for j in range(patches_img.shape[1]):
          single_patch_img = patches_img[i, j, 0, :, :, :]
          if not cv2.imwrite('/content/drive/MyDrive/testpatchi/masks/' + tmp_image_name+'_'+ str(i)+str(j)+'.png', single_patch_img):
            raise Exception("Could not write the image")

dice loss

In [3]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

model


In [4]:


from tensorflow.keras.models import Model
#convolution block
def cblock(input, output):
    x = Conv2D(output, 3, padding="same")(input) #kernel size=3, zero padding 
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
  
    x = Conv2D(output, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

#encoder block
def encoder_block(input, output):
    x = cblock(input, output) 
        
    p = MaxPool2D((2, 2))(x)    #2 x2 pooling layer 
    return x, p #output of conv block and maxpooling layer

#decoder block
def decoder_block(input, skip_output, output):
    x = Conv2DTranspose(output, (2, 2), strides=2, padding="same",)(input)
 
    x = Concatenate()([x, skip_output])
    x = cblock(x, output)
    return x

def unet(input_shape): #encoder-bridge-decoder-final layer
    inputs = Input(input_shape)
#4 encoder layer
    s1, p1 = encoder_block(inputs,32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)
    s4, p4 = encoder_block(p3, 256)
#bridge
    b1 = cblock(p4, 512)
#4 decoder layers
    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)
#classification layer
    final = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, final, name="U-Net")
    return model


if __name__ == "__main__":
    input_shape = (256, 256, 3) #no batchsize specified
    model = unet(input_shape)
   # model.summary()

In [8]:



image_directory = '/content/drive/MyDrive/realpatch/train/new/images/'
mask_directory = '/content/drive/MyDrive/realpatch/train/new/masks/'



image_dataset = []  #for images 
mask_dataset = []  #for masks
#load resize
images = sorted(os.listdir(image_directory))
for i, image_name in enumerate(images):    
  
    image = cv2.imread(image_directory+image_name, 1)
    image = Image.fromarray(image)
   
    image_dataset.append(np.array(image))



masks = sorted(os.listdir(mask_directory))
for i, image_name in enumerate(masks):
  image = cv2.imread(mask_directory+image_name, 0)
  image = Image.fromarray(image)
  mask_dataset.append(np.array(image))


#Normalise images
image_dataset = np.array(image_dataset)/255.
#rescale label
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.


#splitdata into training val
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.05, random_state = 0)



In [10]:
len(X_test)

30

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]
num_labels = 1  #Binary
input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
#Hyperparameters
batch_size = 16
num_epochs=200
l_r=0.005
#load model 
unet_model = unet(input_shape)
unet_model.compile(optimizer=Adam(lr = l_r), loss=dice_coef_loss, 
              metrics=['accuracy', jacard_coef,dice_coef])
unet_data = unet_model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test ), 
                    shuffle=False,
                    epochs=num_epochs)
#save model
unet_model.save('Unet.h5')

In [12]:

#write loss jacard coeficient to pandas dataframe
unet_history_df = pd.DataFrame(unet_data.history) 

with open('unet_history_df.csv', mode='w') as f:
    unet_history_df.to_csv(f) 

In [ ]:

history = unet_data

#plot the training and validation loss and jaccard coef per epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['jacard_coef']

val_acc = history.history['val_jacard_coef']


plt.plot(epochs, acc, 'y', label='Training Jacard')
plt.plot(epochs, val_acc, 'r', label='Validation Jacard')
plt.title('Training and validation Jacard')
plt.xlabel('Epochs')
plt.ylabel('Jacard')
plt.legend()
plt.show()

In [ ]:
import random
test_img_number = random.randint(0, X_test.shape[0]-1)
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]

test_img_input=np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)

plt.figure(figsize=(16, 8))
plt.subplot(231)

plt.imshow(test_img, cmap='gray')
plt.subplot(232)

plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)


plt.imshow(prediction, cmap='gray')

plt.show()

from tensorflow.keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(ground_truth[:,:,0], prediction)
print("Mean IoU =", IOU_keras.result().numpy())


#Calculate IoU for all test images and average
 
import pandas as pd

IoU_values = []
for img in range(0, X_test.shape[0]):
    temp_img = X_test[img]
    ground_truth=y_test[img]
    temp_img_input=np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
    
    IoU = MeanIoU(num_classes=n_classes)
    IoU.update_state(ground_truth[:,:,0], prediction)
    IoU = IoU.result().numpy()
    IoU_values.append(IoU)


    


df = pd.DataFrame(IoU_values, columns=["IoU"])
df = df[df.IoU != 1.0]    
mean_IoU = df.mean().values
print("Mean IoU is: ", mean_IoU)    
 


Some reference for the code has been used from the following github link:
<bnsreenu> <https://github.com/bnsreenu/python_for_microscopists/blob/master/224_225_226_mito_segm_using_various_unet_models.py> 
